In [1]:
import polyline
import json
import pandas as pd
from shapely.geometry import Point
import folium
import geopandas as gpd
import dotenv
import os
from openrouteservice import convert
from geopy.distance import geodesic
import time
import math

#Importar funciones de peticiones
from petitions import get_vel,get_vel_azure,get_alt,get_route,get_opt_route


In [2]:
dotenv.load_dotenv()
ORS_API_KEY=os.getenv('ORS_API_KEY')
AZURE_API_KEY=os.getenv('AZURE_API_KEY')
AZURE_API_KEY_2=os.getenv('AZURE_API_KEY_2')

In [9]:
# [Lat,lon] format
def generate_map(coords):
    m=folium.Map(location=coords[0], zoom_start=12)
    folium.Marker(coords[0]).add_to(m)
    folium.Marker(coords[-1]).add_to(m)
    folium.PolyLine(
        coords,
        color="blue",
        weight=5,
        opacity=0.8
    ).add_to(m)
    return m

#line=convert.decode_polyline(ors["routes"][0]["geometry"],False)
#generate_map([[lat,lon]for lon,lat in line["coordinates"]])

In [3]:
with open("routes/ors_route.json","r") as f:
        ors = json.load(f);

with open("routes/azure_opt2.geojson","r") as f:
        azure_route = json.load(f);

med = pd.read_json("results/med.json")


In [4]:
def get_matriz_od(m):
    rows = []
    for i, row in m.iterrows():   
        for j,destino in m.iterrows():
            if i != j:
                new_row = row.copy()
                if "estacion" in row["municipio_o"]:
                    new_row["hora_o"] = destino["hora_o"]
                    new_row["hora_d"] = destino["hora_d"]
                    new_row["modo_tte"] = destino["modo_tte"]
                    new_row["motivo_viaje"] = destino["motivo_viaje"]
                    
                new_row["municipio_d"] = destino["municipio_o"]
                new_row["comuna_d"] = destino["comuna_o"]
                new_row["d_long"] = destino["o_long"]
                new_row["d_lat"] = destino["o_lat"]
                rows.append(new_row)
    return pd.DataFrame(rows, index=[i for i in range(len(rows))])

In [ ]:
i = 0
#for row in medj[:500]:
#    print(i)
#    i = i + 1
#    if(row.get("route") is not None):
#        continue
#    else:
#        try:
#            coord = [[row["o_long"],row["o_lat"]],[row["d_long"],row["d_lat"]]]
#            # Conseguir la ruta con ORS
#            data = get_route(coord,ORS_API_KEY)
#
#            route_line = data["routes"][0]["geometry"]
#
#            route_data=convert.decode_polyline(route_line, True)["coordinates"]
#            
#            steps = data["routes"][0]["segments"][0]["steps"]
#            
#            # Conseguir la ruta con tráfico
#            data_opt = get_opt_route(coord,row["hora_o"],AZURE_API_KEY)
#            data_opt = data_opt["features"]
#
#            line_coords = data_opt[-1]["geometry"]["coordinates"][0]
#            elevation = get_alt(line_coords,ORS_API_KEY)
#            
#            row["route"] = get_vel(steps,route_data)
#            row["route_opt"] = get_vel_azure(data_opt,elevation)
#        except Exception as e:
#            print(e)

In [5]:
o_long = [
    -75.587470,
    -75.587470,
    -75.614743,
    -75.587470,
    -75.551106
]

o_lat = [
        6.213373,
        6.240400,
        6.258418,
        6.285445,
        6.294454
]

estaciones = ["estacion " + str(i) for i in range(1,len(o_lat)+1)]

horas = ["08:00:00" for _ in range(len(o_lat))]


In [6]:
estaciones_df = pd.DataFrame(columns=med.columns)
estaciones_df["o_long"] = o_long
estaciones_df["o_lat"] = o_lat  
estaciones_df["municipio_o"] = estaciones
estaciones_df["comuna_o"] = estaciones
estaciones_df["hora_o"] = horas

In [14]:
estaciones_df

,municipio_o,comuna_o,hora_o,municipio_d,comuna_d,hora_d,motivo_viaje,modo_tte,o_long,o_lat,d_long,d_lat
0,estacion 1,estacion 1,08:00:00,NaN,NaN,NaN,NaN,NaN,-75.587470,6.213373,NaN,NaN
1,estacion 2,estacion 2,08:00:00,NaN,NaN,NaN,NaN,NaN,-75.587470,6.240400,NaN,NaN
2,estacion 3,estacion 3,08:00:00,NaN,NaN,NaN,NaN,NaN,-75.614743,6.258418,NaN,NaN
3,estacion 4,estacion 4,08:00:00,NaN,NaN,NaN,NaN,NaN,-75.587470,6.285445,NaN,NaN
4,estacion 5,estacion 5,08:00:00,NaN,NaN,NaN,NaN,NaN,-75.551106,6.294454,NaN,NaN


In [2]:
matrix = pd.read_json("matrix/amva_routes.json")

In [18]:
matrix

,municipio_o,comuna_o,hora_o,municipio_d,comuna_d,hora_d,motivo_viaje,modo_tte,o_long,o_lat,d_long,d_lat,slopes,coords,speeds
0,Medellin,Belen,17:00:00,Barbosa,Urbana Barbosa,17:15:00,Casa,Moto (Acompa√±ante),-75.594047,6.222594,-75.371796,6.415099,"[0, -3.4, 0.0, 0.0, -4.01, 0.0, 0.0, 0.0, -0.9...","[[[-75.59403, 6.22269, 1504.5], [-75.59321, 6....","[[4.17, 4.15, 5.57, 5.57, 5.57, 5.57, 5.57, 12..."
1,Medellin,Belen,17:00:00,Barbosa,Urbana Barbosa,17:15:00,Casa,Moto (Acompa√±ante),-75.594047,6.222594,-75.347011,6.429490,"[0, -3.4, 0.0, 0.0, -4.01, 0.0, 0.0, 0.0, -0.9...","[[-75.59403, 6.22269, 1504.5], [-75.59321, 6.2...","[4.17, 4.15, 5.57, 5.57, 5.57, 5.57, 5.57, 12...."
2,Medellin,Belen,05:45:00,Barbosa,Urbana Barbosa,06:00:00,Trabajo,Moto (Conductor),-75.591182,6.230922,-75.347011,6.429490,"[0, 9.86, 0.0, 0.0, 0.0, 0.0, -14.49, 0.0, 0.0...","[[-75.59118, 6.23099, 1499.0], [-75.59144, 6.2...","[4.14, 7.66, 7.66, 7.66, 7.66, 7.66, 7.66, 7.6..."
3,Medellin,Belen,05:45:00,Barbosa,Urbana Barbosa,06:00:00,Trabajo,Moto (Conductor),-75.591182,6.230922,-75.371796,6.415099,"[0, 9.86, 0.0, 0.0, 0.0, 0.0, -14.49, 0.0, 0.0...","[[-75.59118, 6.23099, 1499.0], [-75.59144, 6.2...","[4.14, 7.66, 7.66, 7.66, 7.66, 7.66, 7.66, 7.6..."
4,Medellin,Belen,17:00:00,Bello,Bello-Comuna 02,17:15:00,Casa,Moto (Acompa√±ante),-75.594047,6.222594,-75.583397,6.316436,"[0, -3.4, 0.0, 0.0, -4.01, 0.0, 0.0, 0.0, -0.9...","[[-75.59403, 6.22269, 1504.5], [-75.59321, 6.2...","[4.17, 4.15, 5.57, 5.57, 5.57, 5.57, 5.57, 12...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,estacion 9,estacion 9,08:00:00,estacion 4,estacion 4,None,None,None,-75.560197,6.334109,-75.569288,6.207983,"[0, -24.23, -4.46, 1.08, 0.0, 0.0, -13.9, 0.0,...","[[-75.56019, 6.33419, 1460.5], [-75.56008, 6.3...","[4.15, 6.87, 6.87, 6.87, 6.87, 6.87, 6.87, 6.8..."
866,estacion 9,estacion 9,08:00:00,estacion 5,estacion 5,None,None,None,-75.560197,6.334109,-75.623833,6.253028,"[0, -24.23, -4.46, 1.08, 0.0, 0.0, -13.9, 0.0,...","[[-75.56019, 6.33419, 1460.5], [-75.56008, 6.3...","[4.15, 6.87, 6.87, 6.87, 6.87, 6.87, 6.87, 6.8..."
867,estacion 9,estacion 9,08:00:00,estacion 6,estacion 6,None,None,None,-75.560197,6.334109,-75.569288,6.253028,"[0, -24.23, -4.46, 1.08, 0.0, 0.0, -13.9, 0.0,...","[[-75.56019, 6.33419, 1460.5], [-75.56008, 6.3...","[4.15, 6.87, 6.87, 6.87, 6.87, 6.87, 6.87, 6.8..."
868,estacion 9,estacion 9,08:00:00,estacion 7,estacion 7,None,None,None,-75.560197,6.334109,-75.587470,6.262037,"[0, -24.23, -4.46, 1.08, 0.0, 0.0, -13.9, 0.0,...","[[-75.56019, 6.33419, 1460.5], [-75.56008, 6.3...","[4.15, 6.87, 6.87, 6.87, 6.87, 6.87, 6.87, 6.8..."


In [11]:
coords = [[lat,lon] for lon,lat,alt in matrix.iloc[0]["coords"][0]]

In [14]:
m = generate_map(coords)

In [16]:
m.save("ruta.html")

In [ ]:
#Función para añadir cuando la matriz está parcialmente completa

vistos = {}
for i, row in matrix.iterrows():
    if(vistos.get(row["o_lat"]) != None):
        continue
    vistos[row["o_lat"]] = 1
    for j, estacion in estaciones_df.iterrows():
        if(row["o_lat"] == estacion["o_lat"] and row["o_long"] == estacion["o_long"]):
            continue
        new_row = row.copy()
        new_row["municipio_d"] = estacion["municipio_o"]
        new_row["comuna_d"] = estacion["comuna_o"]
        new_row["d_long"] = estacion["o_long"]
        new_row["d_lat"] = estacion["o_lat"]
        new_row[["slopes","speeds","coords"]] = None
        matrix.loc[len(matrix)] = new_row

        #Crear ruat inversa
        new_row = estacion.copy()
        new_row["municipio_d"] = row["municipio_o"]
        new_row["comuna_d"] = row["comuna_o"]
        new_row["d_long"] = row["o_long"]
        new_row["d_lat"] = row["o_lat"]
        matrix.loc[len(matrix)] = new_row
        

In [7]:

df = get_matriz_od(estaciones_df)

In [33]:
matrix = pd.concat([matrix, df], ignore_index=True)

In [8]:
comunas = med["comuna_o"].unique()
m = pd.DataFrame(columns=med.columns)
x=9
for i in range(len(comunas)):
    m.loc[len(m)] = med[med["comuna_o"]==comunas[i]].iloc[x].values
    m.loc[len(m)] = med[med["comuna_o"]==comunas[i]].iloc[x+1].values

In [9]:
matrix = pd.concat([m,estaciones_df],ignore_index=True)

In [10]:
expand_m = get_matriz_od(matrix)

In [11]:
expand_m

,municipio_o,comuna_o,hora_o,municipio_d,comuna_d,hora_d,motivo_viaje,modo_tte,o_long,o_lat,d_long,d_lat
0,Medellin,La Candelaria,17:30:00,Medellin,La Candelaria,18:00:00,Casa,Moto (Conductor),-75.568624,6.259739,-75.568943,6.256038
1,Medellin,La Candelaria,17:30:00,Medellin,Guayabal,18:00:00,Casa,Moto (Conductor),-75.568624,6.259739,-75.587775,6.226936
2,Medellin,La Candelaria,17:30:00,Medellin,Guayabal,18:00:00,Casa,Moto (Conductor),-75.568624,6.259739,-75.590442,6.225021
3,Medellin,La Candelaria,17:30:00,Medellin,El Poblado,18:00:00,Casa,Moto (Conductor),-75.568624,6.259739,-75.569991,6.188184
4,Medellin,La Candelaria,17:30:00,Medellin,El Poblado,18:00:00,Casa,Moto (Conductor),-75.568624,6.259739,-75.559199,6.207943
...,...,...,...,...,...,...,...,...,...,...,...,...
2157,estacion 5,estacion 5,05:30:00,Medellin,Santa Cruz,06:00:00,Estudio,Moto (Conductor),-75.551106,6.294454,-75.557309,6.309614
2158,estacion 5,estacion 5,08:00:00,estacion 1,estacion 1,NaN,NaN,NaN,-75.551106,6.294454,-75.587470,6.213373
2159,estacion 5,estacion 5,08:00:00,estacion 2,estacion 2,NaN,NaN,NaN,-75.551106,6.294454,-75.587470,6.240400
2160,estacion 5,estacion 5,08:00:00,estacion 3,estacion 3,NaN,NaN,NaN,-75.551106,6.294454,-75.614743,6.258418


In [12]:
expand_m['slopes'] = None
expand_m['coords'] = None
expand_m['speeds'] = None

In [16]:
expand_m = pd.read_json("matrix/amva_traffic.json")

In [ ]:
# Obtener altitudes cada dos puntos para rutas con trafico
for i in range(0, len(expand_m), 2):
    print(i)
    rows = expand_m.iloc[i:i+2]

    coords_list = []
    row_indices = []

    for idx, row in rows.iterrows():
        if expand_m.at[idx, "slopes"] is not None:
            continue

        coord = [[row["o_long"], row["o_lat"]], [row["d_long"], row["d_lat"]]]

        data = get_opt_route(coord,row["hora_o"] ,AZURE_API_KEY_2)
        route_data = data["features"][-1]["geometry"]["coordinates"][0]

        coords_list.append((idx, route_data, data["features"][:-1]))
        row_indices.append(idx)

    if not coords_list:
        print("lop")
        continue

    lop = []
    for _, route_data, _ in coords_list:
        lop.extend(route_data)

    pol = get_alt(lop, ORS_API_KEY)

    cursor = 0
    for idx, route_data, steps in coords_list:
        seg_len = len(route_data)
        coords = pol[cursor: cursor + seg_len]
        cursor += seg_len

        route = get_vel_azure(steps, coords)

        expand_m.at[idx, "slopes"] = route["slope_deg"]
        expand_m.at[idx, "coords"] = route["coords"]
        expand_m.at[idx, "speeds"] = route["speed_ms"]

    time.sleep(1.5)


In [222]:
idx = 657

if expand_m.at[idx, "slopes"] is None:
    row = expand_m.iloc[idx]
    coord = [[row["o_long"], row["o_lat"]], [row["d_long"], row["d_lat"]]]
    
    data = get_opt_route(coord,row["hora_o"] ,AZURE_API_KEY_2)
    route_data = data["features"][-1]["geometry"]["coordinates"][0]

    elevation = get_alt(route_data, ORS_API_KEY)
    steps = data["features"][:-1]
    route = get_vel_azure(steps, elevation)

    expand_m.at[idx, "slopes"] = route["slope_deg"]
    expand_m.at[idx, "coords"] = route["coords"]
    expand_m.at[idx, "speeds"] = route["speed_ms"]

In [23]:
expand_m.to_json("matrix/amva_traffic.json",orient="records")

In [13]:
matrix = expand_m

In [3]:
matrix = pd.read_json("matrix/med.json")

In [17]:
for idx, row in matrix.iterrows():
    print(idx)
    if matrix.iloc[idx]['slopes'] is not None and not matrix.iloc[idx].isna()["slopes"]:
        print("lop")
        continue
    else:
        coord = [[row["o_long"], row["o_lat"]], [row["d_long"], row["d_lat"]]]
        # Get route with ORS
        data = get_route(coord, ORS_API_KEY)
        route_line = data["routes"][0]["geometry"]
        route_data = convert.decode_polyline(route_line, True)["coordinates"]
        
        steps = data["routes"][0]["segments"][0]["steps"]
        route = get_vel(steps, route_data)
        
        matrix.at[idx, "slopes"] = [route["slope_deg"]]
        matrix.at[idx, "coords"] = [route["coords"]]
        matrix.at[idx, "speeds"] = [route["speed_ms"]]
        time.sleep(2.5)


0
lop
1
lop
2
lop
3
lop
4
lop
5
lop
6
lop
7
lop
8
lop
9
lop
10
lop
11
lop
12
lop
13
lop
14
lop
15
lop
16
lop
17
lop
18
lop
19
lop
20
lop
21
lop
22
lop
23
lop
24
lop
25
lop
26
lop
27
lop
28
lop
29
lop
30
lop
31
lop
32
lop
33
lop
34
lop
35
lop
36
lop
37
lop
38
lop
39
lop
40
lop
41
lop
42
lop
43
lop
44
lop
45
lop
46
lop
47
lop
48
lop
49
lop
50
lop
51
lop
52
lop
53
lop
54
lop
55
lop
56
lop
57
lop
58
lop
59
lop
60
lop
61
lop
62
lop
63
lop
64
lop
65
lop
66
lop
67
lop
68
lop
69
lop
70
lop
71
lop
72
lop
73
lop
74
lop
75
lop
76
lop
77
lop
78
lop
79
lop
80
lop
81
lop
82
lop
83
lop
84
lop
85
lop
86
lop
87
lop
88
lop
89
lop
90
lop
91
lop
92
lop
93
lop
94
lop
95
lop
96
lop
97
lop
98
lop
99
lop
100
lop
101
lop
102
lop
103
lop
104
lop
105
lop
106
lop
107
lop
108
lop
109
lop
110
lop
111
lop
112
lop
113
lop
114
lop
115
lop
116
lop
117
lop
118
lop
119
lop
120
lop
121
lop
122
lop
123
lop
124
lop
125
lop
126
lop
127
lop
128
lop
129
lop
130
lop
131
lop
132
lop
133
lop
134
lop
135
lop
136
lop
137
lop
138
lo

KeyboardInterrupt: 

In [18]:
matrix.to_json("matrix/med.json")

In [19]:
lop = pd.read_json("matrix/med.json")

In [20]:
lop.iloc[1850:]

,municipio_o,comuna_o,hora_o,municipio_d,comuna_d,hora_d,motivo_viaje,modo_tte,o_long,o_lat,d_long,d_lat,slopes,coords,speeds
1850,Medellin,Santa Cruz,10:00:00,Medellin,Popular,10:20:00,Trabajo,Moto (Conductor),-75.555627,6.300949,-75.541509,6.290188,"[[0, 17.81, 0.0, 0.0, 3.92, 1.87, 11.21, 0.0, ...","[[[-75.55563, 6.30083, 1464.3], [-75.55547, 6....","[[4.16, 4.16, 10.95, 4.16, 4.16, 12.52, 12.52,..."
1851,Medellin,Santa Cruz,10:00:00,Medellin,Popular,10:20:00,Trabajo,Moto (Conductor),-75.555627,6.300949,-75.537273,6.293613,None,None,None
1852,Medellin,Santa Cruz,10:00:00,Medellin,San Javier,10:20:00,Trabajo,Moto (Conductor),-75.555627,6.300949,-75.620456,6.262199,None,None,None
1853,Medellin,Santa Cruz,10:00:00,Medellin,San Javier,10:20:00,Trabajo,Moto (Conductor),-75.555627,6.300949,-75.618034,6.258945,None,None,None
1854,Medellin,Santa Cruz,10:00:00,Medellin,Laureles-Estadio,10:20:00,Trabajo,Moto (Conductor),-75.555627,6.300949,-75.607598,6.241542,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,estacion 5,estacion 5,05:30:00,Medellin,Santa Cruz,06:00:00,Estudio,Moto (Conductor),-75.551106,6.294454,-75.557309,6.309614,None,None,None
2158,estacion 5,estacion 5,08:00:00,estacion 1,estacion 1,None,None,None,-75.551106,6.294454,-75.587470,6.213373,None,None,None
2159,estacion 5,estacion 5,08:00:00,estacion 2,estacion 2,None,None,None,-75.551106,6.294454,-75.587470,6.240400,None,None,None
2160,estacion 5,estacion 5,08:00:00,estacion 3,estacion 3,None,None,None,-75.551106,6.294454,-75.614743,6.258418,None,None,None


In [119]:
for i, row in matrix[1:].iterrows():
    matrix.at[i,"slopes"] = row["slopes"][0]
    matrix.at[i,"speeds"] = row["speeds"][0]
    matrix.at[i,"coords"] = row["coords"][0]

In [121]:
matrix.to_json("matrix/amva_routes.json")

In [122]:
m1 = pd.read_json("matrix/amva_routes.json")